In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
processed_data = np.load('Full_array.npy')

from tensorflow import keras

import pandas as pd

#The size of the dataset is 10,000; but I only filled the first 9166 with values, the rest are zeros. (shouldn't be counted)
number = 9166
processed_data = processed_data[0:number]

np.random.seed(1)
tf.set_random_seed(1)

In [2]:
#we want to normalize our iamge data to go from 0 to 1
processed_data = processed_data.reshape(np.size(processed_data))
processed_data = (processed_data - min(processed_data))/ (max(processed_data) - min(processed_data))
processed_data = processed_data.reshape(number,28,28)
#I actually am not sure how it is scaled. however, some of the pixels are negative for some reason???

#also, this did not help.

In [3]:
#hyperparameters:
HP_allowed_confidence = 0.98
test_train_split = 0.85
batch_size = 64
epoch_number = 5
learning_rate = 0.001
dropout_rate = 0.3
CNL1_filters = 32
CNL1_kernal_size = 3
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 3
MPL2_pool_size = (2,2)
MPL2_strides = 2

#I have a non-round number of examples
train_split_indice = int(np.round(test_train_split*number))

In [4]:
#Load in my dataset of targets, targets are strings labels under the name "Class"
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[2,3,4,8,15,21,27], nrows=number)
#galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", nrows=number)
Class = galaxyzoo["gz2class"].values
RA = galaxyzoo['ra'].values
DEC = galaxyzoo['dec'].values
Spiral = galaxyzoo['t01_smooth_or_features_a02_features_or_disk_debiased'].values
Elliptical = galaxyzoo['t01_smooth_or_features_a01_smooth_debiased'].values
Anythingelse = galaxyzoo['t01_smooth_or_features_a03_star_or_artifact_debiased'].values

In [5]:
#I want to take the first character of the Class string and interpret as a integer, ala MNIST example code
dictionary = {'A':int(2),'E':int(1),'S':int(0)}

In [6]:
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [7]:
#go through, find the arrays of zero, set that target to 'A' = 2
for i in range(len(target)):
    if processed_data[i].any() == 0:
        target[i] = 2

In [8]:
#split my data between training and test sets
train_target = target[0:train_split_indice]
test_target = target[train_split_indice:number]
train_images = processed_data[0:train_split_indice]
test_images = processed_data[train_split_indice:number]

In [9]:
#Create 3 more images from each of my training and testing images that is each image but flipped 90 degrees...
train_images_flip1 = np.flip(train_images,1)
train_images_flip2 = np.flip(train_images,2)
train_images_flip3 = np.flip(train_images_flip1,2)

flipped_img = np.append(train_images, train_images_flip1, 0)
flipped_img = np.append(flipped_img, train_images_flip2, 0)
flipped_img = np.append(flipped_img, train_images_flip3, 0)

flipped_tar = np.append(train_target,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)
flipped_tar = np.append(flipped_tar,train_target, 0)

print(np.shape(flipped_tar))
print(np.shape(flipped_img))
print(np.size(train_target)*4)

print(np.shape(test_target))

(31164,)
(31164, 28, 28)
31164
(1375,)


In [10]:
train_images = flipped_img
train_target = flipped_tar

In [11]:
np.shape(train_images[0])

(28, 28)

In [12]:
from PIL import Image
#rotate each image in the target_images by a random amount
rotations_array = np.empty((31164,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)


#every time we run this segment, we increase our training set by a factor of 2. this means it will train in 120 * number of times
# times 20 epochs = a long time.

In [13]:
from cv2 import resize

In [14]:
def cv2_clipped_zoom(img, zoom_factor):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

In [15]:
#increase by a factor of two again, like above
rotations_array = np.empty((31164*2,28,28))
for i in range(np.shape(train_images)[0]):
    degree = np.random.uniform(-45,45,1)
    img = Image.fromarray(train_images[i])
    rot = img.rotate(degree)
    zoom_factor = np.random.uniform(0.7,1.3,1)
    rot = cv2_clipped_zoom(np.asarray(rot),zoom_factor)
    #rot = rot.resize()
    #rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
train_images = np.append(train_images, rotations_array,0)
#now append the class labels to the train_target array
train_target = np.append(train_target,train_target)

In [16]:
#we want to form a subset of our training examples that are probabilities that they are sorted correctly greater than %90
#we will then train this dataset additional times.
train_images = train_images.reshape(train_split_indice*4*2*2,28,28,1)
test_images = test_images.reshape((train_split_indice-number)*4*2*2,28,28,1)

In [17]:
from sklearn.utils import shuffle
#X=[1,2,3]
#y = ['one', 'two', 'three']
#X, y = shuffle(X, y, random_state=0)
#print(X)
#print(y)
train_images, train_target = shuffle(train_images, train_target, random_state=1)


In [18]:
from tensorflow.keras.layers import Activation, Conv2D, Add, InputLayer

In [19]:
#Found this function that defines a residual block to use in the sequential model. I like the sequential model. I will keep my
#training wheels for now and learn functional API later.
class Residual(tf.keras.layers.Layer):
    
    def __init__(self, channels_in,kernel,**kwargs):
        super(Residual, self).__init__(**kwargs)
        self.channels_in = channels_in
        self.kernel = kernel

    def call(self, x):
        # the residual block using Keras functional API
        first_layer =   Activation("linear", trainable=False)(x)
        x =             Conv2D( self.channels_in,
                                self.kernel,
                                padding="same")(first_layer)
        x =             Activation("relu")(x)
        x =             Conv2D( self.channels_in,
                                self.kernel,
                                padding="same")(x)
        residual =      Add()([x, first_layer])
        x =             Activation("relu")(residual)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape
    

#call as model.add( Residual(32, (3,3) ))
#residual layers replace convolutional layers. We get rid of dropout when using a res-net

In [20]:

#define my model, using a CNN with 2 Convolutional layers, 2 max pool layers, 1 dense layer, 1 drop out layer, and another dense layer 
def create_model(dropout_rate=dropout_rate, learning_rate=learning_rate):
    model = keras.Sequential([])
    model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters=CNL1_filters,kernel_size=CNL1_kernal_size,
                                  padding="same",activation=tf.nn.relu))
    model.add(Residual(32,(3,3)))
    model.add(Residual(32,(3,3)))
    model.add(Residual(32,(3,3)))
    model.add(Residual(32,(3,3)))
    #model.add(keras.layers.Conv2D(filters=128,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    #model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    return(model)

In [21]:
#validation_split = int(np.round(np.size(train_target)*0.90))
#validation_images = train_images[validation_split::]
#validation_target = train_target[validation_split::]
#train_target = train_target[0:validation_split]
#train_iamges = train_images[0:validation_split]

In [24]:
model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)

model.load_weights("RES_NET_3_28_19.h5")
adam = tf.train.AdamOptimizer(learning_rate = learning_rate)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [33]:
model.fit(train_images, train_target, epochs=epoch_number, batch_size=batch_size, verbose=1, shuffle=True)
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', val_acc)

Epoch 1/3
124656/124656 [==============================] - 3007s 24ms/step - loss: 0.4314 - acc: 0.8074
Epoch 2/3
124656/124656 [==============================] - 2468s 20ms/step - loss: 0.4292 - acc: 0.8076
Epoch 3/3
1375/1375 [==============================] - 12s 9ms/step
Test accuracy: 0.8196363630728288


In [31]:
#model.summary()
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)

#12 layers

#0.808 after 10 epochs, typically, RESNETs get trained for a lot of epochs, like 50 - 100.

#Implement transfer learning using exception? or any state of the art on imagenet. 
#I should search around and choose a state of the art with the minimum amount of parameters, (hard to train).
#too few images is what professor Huerta describes as the problem, which is interesting to note.

1375/1375 [==============================] - 13s 9ms/step
Test accuracy: 0.821818181254647
